In [2]:
# 1. import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [3]:
# 2. find url and store it in a variable
url = "https://en.wikipedia.org/wiki/List_of_European_cities_by_population_within_city_limits"       



In [4]:
# 3. download html with a get request
response = requests.get(url)
response.status_code # 200 status code means OK!

200

In [5]:
# 4.1. parse html (create the 'soup')
soup = BeautifulSoup(response.content, "html.parser")
# 4.2. check that the html code looks like it should
#soup.prettify

In [6]:
# Select the data required

# Useful link with examples about CSS Selectors with soup: 
#     https://www.crummy.com/software/BeautifulSoup/bs4/doc/#css-selectors


cities = [s.get_text() for s in soup.select('.wikitable tbody tr td:nth-of-type(2)')]
countries = [s.get_text() for s in soup.select('.wikitable tbody tr td:nth-of-type(3)')]
population = [s.get_text() for s in soup.select('.wikitable tbody tr td:nth-of-type(4)')]
coords = [s.get_text() for s in soup.select('.wikitable tbody tr td:nth-of-type(8)')]

In [7]:
# create data frame

# calling DataFrame constructor after zipping
# both lists, with columns specified

df=pd.DataFrame(
    list(zip(cities, countries, population, coords)), columns =['city', 'country', 'population', 'coords']
)
from IPython.core.display import display, HTML

df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   city        32 non-null     object
 1   country     32 non-null     object
 2   population  32 non-null     object
 3   coords      32 non-null     object
dtypes: object(4)
memory usage: 1.1+ KB


C:\Users\All\AppData\Local\Temp\ipykernel_12944\1898521999.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


,city,country,population,coords
0,Istanbul,Turkey\n,"15,840,900[a]\n",41°00′49″N 28°57′18″E﻿ / ﻿41.013611°N 28.955°E...
1,Moscow[b],Russia\n,"12,632,409\n",55°45′00″N 37°37′00″E﻿ / ﻿55.75°N 37.616667°E﻿...
2,London,United Kingdom\n,"9,002,488\n",51°30′26″N 0°07′39″W﻿ / ﻿51.507222°N 0.1275°W﻿...
3,Saint Petersburg,Russia\n,"5,376,672\n",59°57′N 30°18′E﻿ / ﻿59.95°N 30.3°E﻿ / 59.95; 3...
4,Berlin,Germany\n,"3,664,088\n",52°31′00″N 13°23′00″E﻿ / ﻿52.516667°N 13.38333...


In [28]:
df['city'] = df['city'].str.replace("[\(\[].*?[\)\]]", "", regex=True)
df['country'] = df['country'].str.replace("\n", "", regex=True)
df['population'] = df['population'].str.replace("\n", "", regex=True)
df[['kk','ll','ltt']] = df['coords'].str.split('/',expand=True) 
df['kk'] = df['kk'].str.replace("°", ".", regex=True)
df['kk'] = df['kk'].str.replace("′", "", regex=True)
#df.drop(['ll','ltt','latitude'], axis = 1, inplace=True)
df[['latitude','longitude']] = df['kk'].str.split('″N',expand=True) 

In [29]:
df.sample(10)

,city,country,population,coords,kk,longitude,latitude,ll,ltt
0,Istanbul,Turkey,"15,840,900[a]",41°00′49″N 28°57′18″E﻿ / ﻿41.013611°N 28.955°E...,41.0049″N 28.5718″E﻿,28.5718″E﻿,41.0049,﻿41.013611°N 28.955°E﻿,41.013611; 28.955﻿ (1 Istanbul)\n
7,Rome,Italy,"2,844,750",41°54′N 12°30′E﻿ / ﻿41.9°N 12.5°E﻿ / 41.9; 12....,41.54N 12.30E﻿,None,41.54N 12.30E﻿,﻿41.9°N 12.5°E﻿,41.9; 12.5﻿ (8 Rome)\n
3,Saint Petersburg,Russia,"5,376,672",59°57′N 30°18′E﻿ / ﻿59.95°N 30.3°E﻿ / 59.95; 3...,59.57N 30.18E﻿,None,59.57N 30.18E﻿,﻿59.95°N 30.3°E﻿,59.95; 30.3﻿ (4 Saint Petersburg)\n
8,Bucharest,Romania,"2,161,347",44°25′57″N 26°06′14″E﻿ / ﻿44.4325°N 26.103889°...,44.2557″N 26.0614″E﻿,26.0614″E﻿,44.2557,﻿44.4325°N 26.103889°E﻿,44.4325; 26.103889﻿ (10 Bucharest)\n
18,Kharkiv,Ukraine,"1,414,319",50°00′16″N 36°13′53″E﻿ / ﻿50.004444°N 36.23138...,50.0016″N 36.1353″E﻿,36.1353″E﻿,50.0016,﻿50.004444°N 36.231389°E﻿,50.004444; 36.231389﻿ (19 Kharkiv)\n
25,Birmingham,United Kingdom,"1,140,525",52°28′59″N 1°53′37″W﻿ / ﻿52.483056°N 1.893611°...,52.2859″N 1.5337″W﻿,1.5337″W﻿,52.2859,﻿52.483056°N 1.893611°W﻿,52.483056; -1.893611﻿ (26 Birmingham)\n
27,Ufa,Russia,"1,125,933",54°45′00″N 55°58′00″E﻿ / ﻿54.75°N 55.966667°E﻿...,54.4500″N 55.5800″E﻿,55.5800″E﻿,54.4500,﻿54.75°N 55.966667°E﻿,54.75; 55.966667﻿ (28 Ufa)\n
24,Samara,Russia,"1,144,759",53°12′10″N 50°08′27″E﻿ / ﻿53.202778°N 50.14083...,53.1210″N 50.0827″E﻿,50.0827″E﻿,53.1210,﻿53.202778°N 50.140833°E﻿,53.202778; 50.140833﻿ (25 Samara)\n
11,Vienna,Austria,"1,931,830",48°12′00″N 16°22′00″E﻿ / ﻿48.2°N 16.366667°E﻿ ...,48.1200″N 16.2200″E﻿,16.2200″E﻿,48.1200,﻿48.2°N 16.366667°E﻿,48.2; 16.366667﻿ (12 Vienna)\n
30,Perm,Russia,"1,049,199",58°00′00″N 56°19′00″E﻿ / ﻿58°N 56.316667°E﻿ / ...,58.0000″N 56.1900″E﻿,56.1900″E﻿,58.0000,﻿58°N 56.316667°E﻿,58; 56.316667﻿ (31 Perm)\n
